In [1]:
from count_configuration import CountConfiguration
from cached_runner import CachedRunner

configuration = CountConfiguration()
count_runner = CachedRunner(configuration)
reduced_matrixes = count_runner.get_datasets(fresh=True)

processing: /analyzer/input/523436468908064768/545193393672814593/channel.json cache: /analyzer/tmp/523436468908064768/545193393672814593/count_cache_channel.json
processing: /analyzer/input/523436468908064768/557379696472358912/channel.json cache: /analyzer/tmp/523436468908064768/557379696472358912/count_cache_channel.json
processing: /analyzer/input/523436468908064768/550823329544667149/channel.json cache: /analyzer/tmp/523436468908064768/550823329544667149/count_cache_channel.json
processing: /analyzer/input/523436468908064768/538662191998697472/channel.json cache: /analyzer/tmp/523436468908064768/538662191998697472/count_cache_channel.json
processing: /analyzer/input/523436468908064768/559695674384318464/channel.json cache: /analyzer/tmp/523436468908064768/559695674384318464/count_cache_channel.json
processing: /analyzer/input/523436468908064768/542288295682375681/channel.json cache: /analyzer/tmp/523436468908064768/542288295682375681/count_cache_channel.json
processing: /analyzer/

processing: /analyzer/input/523436468908064768/688441819532754952/channel.json cache: /analyzer/tmp/523436468908064768/688441819532754952/count_cache_channel.json
processing: /analyzer/input/523436468908064768/557456044629164033/channel.json cache: /analyzer/tmp/523436468908064768/557456044629164033/count_cache_channel.json
processing: /analyzer/input/523436468908064768/567348322562736148/channel.json cache: /analyzer/tmp/523436468908064768/567348322562736148/count_cache_channel.json
processing: /analyzer/input/523436468908064768/538330246349324289/channel.json cache: /analyzer/tmp/523436468908064768/538330246349324289/count_cache_channel.json
processing: /analyzer/input/523436468908064768/538332551496859650/channel.json cache: /analyzer/tmp/523436468908064768/538332551496859650/count_cache_channel.json
processing: /analyzer/input/523436468908064768/552516649291612161/channel.json cache: /analyzer/tmp/523436468908064768/552516649291612161/count_cache_channel.json
processing: /analyzer/

processing: /analyzer/input/742745493570060302/751336247733256232/channel.json cache: /analyzer/tmp/742745493570060302/751336247733256232/count_cache_channel.json
processing: /analyzer/input/742745493570060302/753932659461455942/channel.json cache: /analyzer/tmp/742745493570060302/753932659461455942/count_cache_channel.json
processing: /analyzer/input/742745493570060302/760206996296564745/channel.json cache: /analyzer/tmp/742745493570060302/760206996296564745/count_cache_channel.json
processing: /analyzer/input/742745493570060302/754941297760010270/channel.json cache: /analyzer/tmp/742745493570060302/754941297760010270/count_cache_channel.json
processing: /analyzer/input/742745493570060302/755506573639418026/channel.json cache: /analyzer/tmp/742745493570060302/755506573639418026/count_cache_channel.json
processing: /analyzer/input/742745493570060302/770058103877664778/channel.json cache: /analyzer/tmp/742745493570060302/770058103877664778/count_cache_channel.json
processing: /analyzer/

processing: /analyzer/input/610782839805444126/620694223901032449/channel.json cache: /analyzer/tmp/610782839805444126/620694223901032449/count_cache_channel.json
processing: /analyzer/input/610782839805444126/618441963372544026/channel.json cache: /analyzer/tmp/610782839805444126/618441963372544026/count_cache_channel.json
processing: /analyzer/input/610782839805444126/640838601290154014/channel.json cache: /analyzer/tmp/610782839805444126/640838601290154014/count_cache_channel.json
processing: /analyzer/input/610782839805444126/625029505471217682/channel.json cache: /analyzer/tmp/610782839805444126/625029505471217682/count_cache_channel.json
processing: /analyzer/input/610782839805444126/639264565644754981/channel.json cache: /analyzer/tmp/610782839805444126/639264565644754981/count_cache_channel.json
processing: /analyzer/input/610782839805444126/625576955684585482/channel.json cache: /analyzer/tmp/610782839805444126/625576955684585482/count_cache_channel.json
processing: /analyzer/

processing: /analyzer/input/610782839805444126/640852795154759680/channel.json cache: /analyzer/tmp/610782839805444126/640852795154759680/count_cache_channel.json
processing: /analyzer/input/610782839805444126/614571380201095238/channel.json cache: /analyzer/tmp/610782839805444126/614571380201095238/count_cache_channel.json
processing: /analyzer/input/610782839805444126/613840014270398476/channel.json cache: /analyzer/tmp/610782839805444126/613840014270398476/count_cache_channel.json
processing: /analyzer/input/610782839805444126/646326463167201280/channel.json cache: /analyzer/tmp/610782839805444126/646326463167201280/count_cache_channel.json
processing: /analyzer/input/610782839805444126/641783762769936417/channel.json cache: /analyzer/tmp/610782839805444126/641783762769936417/count_cache_channel.json
processing: /analyzer/input/610782839805444126/633439569496375307/channel.json cache: /analyzer/tmp/610782839805444126/633439569496375307/count_cache_channel.json
processing: /analyzer/

...

With group: python 2019
With elements: ['EX / ex01', 'EX / ex02', 'EX / ex03_idcode', 'EX / ex04_cipher', 'EX / ex05_pies', 'EX / ex06_schedule', 'EX / ex07_minesweeper', 'EX / ex08_solution_and_test', 'EX / ex09_recursive_calories', 'EX / ex11_order', 'EX / ex12_adventure', 'EX / ex13_blackjack', 'EX / ex14_pokemon', 'EX / ex15_magic', 'fun / aoc', 'fun / food', 'fun / gaming', 'fun / meme', 'fun / music', 'fun / random', 'fun / wat', 'general / eksam', 'general / general', 'general / hääletused', 'general / maintenance', 'general / pydoc', 'general / stat', 'general / teated', 'general / ülesanded', 'Õppegrupid / iaab', 'Õppegrupid / iadb', 'Õppegrupid / iaib', 'Õppegrupid / kaugõpe', 'PR / pr01', 'PR / pr02-primes', 'PR / pr03-rps', 'PR / pr04-filter', 'PR / pr06_regex', 'PR / pr07_pyramid', 'PR / pr08_testing', 'PR / pr09_recursion', 'PR / pr11_drawing', 'PR / pr12_pizzeria', 'PR / pr13_api', 'PR / pr14_exam', 'PR / pr15_lambda', 'täiendusõpe / t01', 'täiendusõpe / t02', 'täiendusõ

With group: python 2019 EX
With elements: ['EX / ex01', 'EX / ex02', 'EX / ex03_idcode', 'EX / ex04_cipher', 'EX / ex05_pies', 'EX / ex06_schedule', 'EX / ex07_minesweeper', 'EX / ex08_solution_and_test', 'EX / ex09_recursive_calories', 'EX / ex11_order', 'EX / ex12_adventure', 'EX / ex13_blackjack', 'EX / ex14_pokemon', 'EX / ex15_magic']
With group: Java 2020 Text Channels
With elements: ['Text Channels / food', 'Text Channels / general', 'Text Channels / gomoku', 'Text Channels / logo', 'Text Channels / meme', 'Text Channels / projekt', 'Text Channels / random', 'Text Channels / robocode', 'Text Channels / teated']
With group: Python 2020 PR
With elements: ['PR / pr01_python', 'PR / pr02_primes', 'PR / pr03_pancakes', 'PR / pr04_census', 'PR / pr06_twitter', 'PR / pr07_train', 'PR / pr08_secret_garden', 'PR / pr09_inception', 'PR / pr11_shapes', 'PR / pr12_zoo', 'PR / pr13_google_api', 'PR / pr14_robot_api']
With group: java 2019 Kalmo
With elements: ['Kalmo / hw03', 'Kalmo / hw04',

With group: general / teated
With elements: ['general / teated']
With group: Text Channels / kaitsmine
With elements: ['Text Channels / kaitsmine']
With group: Text Channels / wat
With elements: ['Text Channels / wat']
With group: EX / ex07-typegame
With elements: ['EX / ex07-typegame']
With group: Text Channels / näidis-tk
With elements: ['Text Channels / näidis-tk']
With group: Kalmo / hw05
With elements: ['Kalmo / hw05']
With group: EX / ex12_adventure
With elements: ['EX / ex12_adventure']
With group: ex / ex02-webbrowser
With elements: ['ex / ex02-webbrowser']
With group: general / ülesanded
With elements: ['general / ülesanded']
With group: PR / pr12-rekursioon
With elements: ['PR / pr12-rekursioon']
With group: EX / ex01-id-code
With elements: ['EX / ex01-id-code']
With group: general / pydoc
With elements: ['general / pydoc']
With group: xp / xp01_janguru
With elements: ['xp / xp01_janguru']
With group: xp / 11450-wedding-shopping
With elements: ['xp / 11450-wedding-shopping']


With group: FUN / games
With elements: ['FUN / games']
With group: täiendusõpe / t02
With elements: ['täiendusõpe / t02']
With group: täiendusõpe / t08
With elements: ['täiendusõpe / t08']
With group: fun / vegan-blog
With elements: ['fun / vegan-blog']
With group: Kalmo / hw07
With elements: ['Kalmo / hw07']
With group: TUGIGRUPID / nele-tugigrupp
With elements: ['TUGIGRUPID / nele-tugigrupp']
With group: PR / pr01_python
With elements: ['PR / pr01_python']
With group: PR / pr01
With elements: ['PR / pr01']
With group: Text Channels / по-русски
With elements: ['Text Channels / по-русски']
With group: EX / ex05_pies
With elements: ['EX / ex05_pies']
With group: PR / pr14_robot_api
With elements: ['PR / pr14_robot_api']
With group: FUN / cats
With elements: ['FUN / cats']
With group: TUGIGRUPID / kaspar-tugigrupp
With elements: ['TUGIGRUPID / kaspar-tugigrupp']
With group: PR / pr11_drawing
With elements: ['PR / pr11_drawing']
With group: PR / pr00-hello
With elements: ['PR / pr00-hello

With group: xp
With elements: ['xp / dj01_intro', 'xp / dj02_django_blog', 'xp / xp01_janguru', 'xp / xp02_conversation', 'xp / xp03_neuralnetwork', 'xp / xp04_heatmap', 'xp / xp05_fractal', 'xp / xp06_maze', 'xp / xp07_strategy', 'xp / xp08_investor', 'xp / 11450-wedding-shopping', 'xp / dj01_django_intro', 'xp / hackerrank', 'xp / xp01_janguru', 'xp / xp02_conversation', 'xp / xp03_fractal', 'xp / xp04_maze', 'xp / xp05_skynet', 'xp / xp06_investor', 'xp / xp07_stargate', 'xp / xp08_sentence_generator']
With group: täiendusõpe
With elements: ['täiendusõpe / t01', 'täiendusõpe / t02', 'täiendusõpe / t03', 'täiendusõpe / t04', 'täiendusõpe / t05', 'täiendusõpe / t06', 'täiendusõpe / t07', 'täiendusõpe / t08', 'täiendusõpe / t09']
With group: Õppegrupid
With elements: ['Õppegrupid / iaab', 'Õppegrupid / iadb', 'Õppegrupid / iaib', 'Õppegrupid / kaugõpe']


In [2]:
print(len(reduced_matrixes))

5
